<a href="https://colab.research.google.com/github/winterForestStump/thesis/blob/main/retrieval/Retrievals_l2_with_reranker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Running on GPU

In [ ]:
!pip install langchain langchain-core langchain-community --quiet
!pip install sentence_transformers FlagEmbedding chromadb --quiet

In [ ]:
import pandas as pd
from langchain.vectorstores import Chroma
from langchain.retrievers import ParentDocumentRetriever
import chromadb
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.storage import InMemoryStore
from langchain.storage._lc_store import create_kv_docstore
from langchain.storage.file_system import LocalFileStore
from FlagEmbedding import FlagReranker
from tqdm import tqdm
import os

In [ ]:
# Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
model_name = "BAAI/bge-small-en-v1.5"
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

bge_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cuda'},
    encode_kwargs=encode_kwargs
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
persistent_client = chromadb.PersistentClient('/content/drive/MyDrive/Thesis/chromadb')
fs = LocalFileStore('/content/drive/MyDrive/Thesis/reports_store_location')
store = create_kv_docstore(fs)

collection = persistent_client.get_or_create_collection('reports_l2')
vectorstore = Chroma(client = persistent_client, collection_name='reports_l2', embedding_function=bge_embeddings,
                    persist_directory='/content/drive/MyDrive/Thesis/chromadb')
vectorstore.persist()

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [ ]:
reranker = FlagReranker('BAAI/bge-reranker-large', use_fp16=True) # Setting use_fp16 to True speeds up computation with a slight performance degradation

In [ ]:
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=256)

In [ ]:
def compute_reranker(company: str, num_parent: int, num_rerank: int):

  '''
  Retrive a lot of parent chunks using company name filter and number of chunks, rerank them with bge-reranker
  '''

  # Get questions
  questions = pd.read_fwf("https://raw.githubusercontent.com/winterForestStump/thesis/main/questions/questions_ver2.txt", names=['question'])

  # Initialize a retriever
  big_chunks_retriever = ParentDocumentRetriever(vectorstore=vectorstore, docstore=store,
                                                 child_splitter=child_splitter, parent_splitter=parent_splitter,
                                                 search_kwargs={'filter': {'company': company}, 'k': num_parent})

  # Invoke retriever without company name in the question
  results_list = []

  for i in tqdm(range(len(questions))):
    approach = f'reranked_{company}'
    query = questions['question'][i]

    response = big_chunks_retriever.invoke(query)

    texts = []
    for j in range(len(response)):
      texts.append([query, response[j].page_content])
    scores = reranker.compute_score(texts)
    combined = list(zip(texts, scores))
    sorted_combined = sorted(combined, key=lambda x: x[1], reverse=True)

    top_texts = [item[0] for item in sorted_combined[:num_rerank]]

    results_list.append(pd.DataFrame({
        'question': query,
        'context': [top_texts]
    }))
  results = pd.concat(results_list, ignore_index=True)

  results.to_json(f'/content/drive/MyDrive/Thesis/retrievals/reranked/results_{approach}.json')

In [ ]:
companies = ['COCA COLA CO', 'AMAZON COM INC', 'PayPal Holdings, Inc.', 'GENERAL MILLS INC', 'Walmart Inc.', 'PEPSICO INC',
             'Kraft Heinz Co', 'Amcor plc', 'Square, Inc.', '3M CO', 'MICROSOFT CORP', 'Ulta Beauty, Inc.', 'AES CORP']

for comp in companies:
  compute_reranker(comp, 10, 2)

100%|██████████| 35/35 [01:54<00:00,  3.27s/it]


In [ ]:
folder_path = '/content/drive/MyDrive/Thesis/retrievals/reranked/'

dataframes = []
for file_name in os.listdir(folder_path):
    if file_name.endswith('.json'):
        approach_name = file_name
        dataframes.append(approach_name)

In [ ]:
len(dataframes)

13